In [1]:
# % pylab inline
from pylab import *

In [2]:
# imports
import os
import cv2
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
def read_images(dir_path):
    
    X = []
    y = []
    
    # define a label map
    labelmap = {
                'airplane': 0,
                'bird': 1,
                'dog': 2,
                'frog': 3,
                'horse': 4,
                'apple': 5,
                'grape': 6,
                'kiwi': 7,
                'lemon': 8,
                'strawberry': 9
               }
    
    directory_list = os.listdir(dir_path)
    # remove OS X's .DS_Store file
    if '.DS_Store' in directory_list:
        directory_list.remove('.DS_Store')
    
    for i, class_name in enumerate(directory_list):
        for j, image_name in enumerate(os.listdir(dir_path+class_name)):
            image_path = dir_path+class_name+'/'+image_name
            image = cv2.imread(image_path)
            X.append(image)
            y.append(labelmap[class_name])
    
    X = np.array(X)
    y = np.array(y)
    
    return X, y

In [4]:
train_dir_path1 = 'hw2 data/data1/train/'
train_dir_path2 = 'hw2 data/data2/train/'

X1, y1 = read_images(train_dir_path1)
X2, y2 = read_images(train_dir_path2)

In [5]:
def load_data(dir_path1, dir_path2):
    
    X1, y1 = read_images(dir_path1)
    X2, y2 = read_images(dir_path2)
    
    X2_resized = np.zeros((X2.shape[0], 32, 32, X2.shape[3]), dtype=np.uint8)
    
    for i in range(X2.shape[0]):
        X2_resized[i,:,:,0] = cv2.resize(X2[i,:,:,0], (32,32))
        X2_resized[i,:,:,1] = cv2.resize(X2[i,:,:,1], (32,32))
        X2_resized[i,:,:,2] = cv2.resize(X2[i,:,:,2], (32,32))
    
    X = np.append(X1, X2_resized, axis=0)
    y = np.append(y1, y2, axis=0)
    
    return X, y

In [6]:
train_dir_path1 = 'hw2 data/data1/train/'
train_dir_path2 = 'hw2 data/data2/train/'
X_train, y_train= load_data(train_dir_path1, train_dir_path2)

In [7]:
class CIFAR10(torch.utils.data.dataset.Dataset):
    __Xs = None
    __ys = None
    
    def __init__(self, dir_path1, dir_path2, transform=None):
        self.transform = transform
        self.__Xs, self.__ys = load_data(dir_path1, dir_path2)
        
    def __getitem__(self, index):
        img = self.__Xs[index]
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if self.transform is not None:
            img = self.transform(img)
            
        # Convert image and label to torch tensors
        img = torch.from_numpy(np.asarray(img))
        label = torch.from_numpy(np.asarray(self.__ys[index]))
        
        return img, label
    
    def __len__(self):
        return self.__Xs.shape[0]

In [8]:
train_dir_path1 = 'hw2 data/data1/train/'
train_dir_path2 = 'hw2 data/data2/train/'
batch_size = 1

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = CIFAR10(train_dir_path1, train_dir_path2, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=1)

In [9]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [10]:
print(type(device))
torch.cuda.empty_cache()

<class 'torch.device'>


In [11]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
if torch.cuda.is_available():
    print("Running on GPU")
    net = net.cuda()

Running on GPU


In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)#, momentum=0.9)

In [19]:
epochs = 50

for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels = data
        if torch.cuda.is_available():
            inputs = inputs.cuda()
            labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs).to(device)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    print('epoch %d/%d \t loss: %.3f' %
              (epoch + 1, epochs, running_loss))
    running_loss = 0.0

print('Finished Training')

epoch 1/50 	 loss: 822.929
epoch 2/50 	 loss: 646.282
epoch 3/50 	 loss: 590.584
epoch 4/50 	 loss: 511.806
epoch 5/50 	 loss: 562.366
epoch 6/50 	 loss: 562.208
epoch 7/50 	 loss: 523.615
epoch 8/50 	 loss: 438.701
epoch 9/50 	 loss: 515.897
epoch 10/50 	 loss: 352.344
epoch 11/50 	 loss: 503.872
epoch 12/50 	 loss: 501.749
epoch 13/50 	 loss: 374.434
epoch 14/50 	 loss: 445.707
epoch 15/50 	 loss: 397.167
epoch 16/50 	 loss: 296.321
epoch 17/50 	 loss: 635.300
epoch 18/50 	 loss: 313.445
epoch 19/50 	 loss: 496.560
epoch 20/50 	 loss: 352.287
epoch 21/50 	 loss: 295.350
epoch 22/50 	 loss: 486.033
epoch 23/50 	 loss: 390.412
epoch 24/50 	 loss: 516.640
epoch 25/50 	 loss: 398.673
epoch 26/50 	 loss: 253.263
epoch 27/50 	 loss: 267.317
epoch 28/50 	 loss: 438.749
epoch 29/50 	 loss: 515.476
epoch 30/50 	 loss: 448.843
epoch 31/50 	 loss: 506.767
epoch 32/50 	 loss: 499.910
epoch 33/50 	 loss: 483.985
epoch 34/50 	 loss: 366.173
epoch 35/50 	 loss: 572.193
epoch 36/50 	 loss: 439.617
e

In [20]:
test_dir_path1 = 'hw2 data/data1/test/'
test_dir_path2 = 'hw2 data/data2/test/'
batch_size = 1

testset = CIFAR10(test_dir_path1, test_dir_path2, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True, num_workers=1)

In [21]:
correct = 0
total = 0

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        if torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the %d test images: %d %%' % (total,
    100 * correct / total))

Accuracy of the network on the 272 test images: 77 %
